In [1]:
%pylab inline
figsize(8,8)
import sys
import rootpath
sys.path.append(rootpath.detect())
from scipy.spatial import distance_matrix

Populating the interactive namespace from numpy and matplotlib


In [2]:
def weak_dominates(Y, x):
    """
    Test whether rows of Y weakly dominate x
    
    Parameters
    ----------
    Y : array_like
        Array of points to be tested. 
        
    x : array_like
        Vector to be tested
        
    Returns
    -------
    c : ndarray (Bool)
        1d-array.  The ith element is True if Y[i] weakly dominates x
    """
    return (Y <= x ).sum(axis=1) == Y.shape[1]


def attainment_sample(Y, Nsamples=1000):
    """
    Return samples from the attainment surface defined by the mutually non-dominating set Y

    Parameters
    ---------
    Y : array_like
        The surface to be sampled. Each row of Y is vector, that is mutually
        with all the other rows of Y
    Nsamples : int
        Number of samples

    Returns
    -------
    S : ndarray
        Array of samples from the attainment surface.
        Shape; Nsamples by Y.shape[1] 
    
    Notes
    -----
    See "Dominance-based multi-objective simulated annealing"
    Kevin Smith, Richard Everson, Jonathan Fieldsend, 
    Chris Murphy, Rashmi Misra.
    IEEE Transactions on Evolutionary Computing. 
    Volume: 12, Issue: 3, June 2008.
    https://ieeexplore.ieee.org/abstract/document/4358782
    """
    N, D = Y.shape
    Ymin = Y.min(axis=0)
    r = Y.max(axis=0) - Ymin
    S = np.zeros((Nsamples, D))
    
    # Set up arrays of the points sorted according to each coordinate.
    Ys = np.zeros((N, D))
    for d in range(D):
        Ys[:,d] = np.sort(Y[:,d])

    for n in range(Nsamples):
        v = np.random.rand(D)*r + Ymin
        m = np.random.randint(D)

        # Bisection search to find the smallest v[m] 
        # so that v is weakly dominated by an element of Y
        lo, hi = 0, N
        while lo < hi:
            mid = (lo+hi)//2
            v[m] = Ys[mid,m]
            if not any(weak_dominates(Y, v)):
                lo = mid+1
            else:
                hi = mid
        if lo == N: lo -= 1
        v[m] = Ys[lo, m]      
        assert lo == N-1 or any(weak_dominates(Y, v))
        S[n,:] = v[:]
    return S


In [3]:
def normalise_to_axes(x, axes=None):
    r = 1
    assert x.ndim == 2
    axes = np.ones(x.shape[1]) if axis is None else np.array(axes)
    
    x_norm = np.zeros_like(x)
    for i, xi in enumerate(x):
        lmbda = sqrt(r**2/np.sum([xi[j]**2/axes[j]**2 for j in range(x.shape[1])]))
        x_norm[i] = xi*lmbda
        
    return x_norm

In [4]:
n_points = 1000
a = 6
b = 1

random normal

In [5]:
%matplotlib qt

In [12]:
fig = figure()
ax0 = fig.add_subplot(2,1,1)

x = random.multivariate_normal([0, 0], np.diag([1**2, 1**2]), size=n_points)
x_norm = normalise_to_axes(x, [1, 1])

ax0.scatter(*x.T, c="C0", alpha=0.3, s=2, label="{} original points".format(n_points))
ax0.scatter(*x_norm.T, c="C1", alpha=0.3, s=2, label= "points projected to unit circle".format(n_points))
ax0.scatter([0],[0], c="C3", label="origin")
ax0.set_aspect('equal')

ax1 = fig.add_subplot(2,1,2, sharex=ax0, sharey=ax0)
x2 = random.multivariate_normal([0, 0], np.diag([a**2, b**2]), size=n_points)
x2_norm = normalise_to_axes(x2, [a, b])

ax1.scatter(*x2.T, c="C0", alpha=0.3, s=5, label="{} original points".format(n_points))
ax1.scatter(*x2_norm.T, c="C1", alpha=0.1, s=8, label = "points projected to elipse".format(n_points))
ax1.scatter([0],[0], c="C3", label="origin")
ax1.set_aspect('equal')

ax0.legend()
ax1.legend()
ax0.set_title("Projection of distribution $\sigma_a=1$, $\sigma_b=1$ \t to circle: $a=1$, $b=1$")
ax1.set_title("Projection of distribution $\sigma_a=6$, $\sigma_b=1$ \t to elipse: $a=6$, $b=1$")

Text(0.5, 1.0, 'Projection of distribution $\\sigma_a=6$, $\\sigma_b=1$ \t to elipse: $a=6$, $b=1$')

In [8]:
fig = figure()
ax0 = fig.add_subplot(2,1,1)

x = random.multivariate_normal([0, 0], np.diag([1**2, 1**2]), size=n_points)
x_norm = normalise_to_axes(x, [1, 1])

ax0.scatter(*abs(x).T, c="C0", alpha=0.3, s=2, label="{} original points".format(n_points))
ax0.scatter(*abs(x_norm).T, c="C1", alpha=0.3, s=2, label= "points projected to unit circle".format(n_points))
ax0.scatter([0],[0], c="C3", label="origin")
ax0.set_aspect('equal')

ax1 = fig.add_subplot(2,1,2, sharex=ax0, sharey=ax0)
x2 = random.multivariate_normal([0, 0], np.diag([a**2, b**2]), size=n_points)
x2_norm = normalise_to_axes(x2, [a, b])

ax1.scatter(*abs(x2).T, c="C0", alpha=0.3, s=5, label="{} original points".format(n_points))
ax1.scatter(*abs(x2_norm).T, c="C1", alpha=0.1, s=8, label = "points projected to elipse".format(n_points))
ax1.scatter([0],[0], c="C3", label="origin")
ax1.set_aspect('equal')

ax0.legend()
ax1.legend()
ax0.set_title("Projection of distribution $\sigma_a=1$, $\sigma_b=1$ \t to circle: $a=1$, $b=1$")
ax1.set_title("Projection of distribution $\sigma_a=6$, $\sigma_b=1$ \t to elipse: $a=6$, $b=1$")

ax0.set_xlim(0, ax0.get_xlim()[1]); ax0.set_ylim(0, ax0.get_ylim()[1])
ax1.set_xlim(0, ax1.get_xlim()[1]); ax1.set_ylim(0, ax1.get_ylim()[1])

(0.0, 3.408006952559815)

In [8]:
fig = figure()
ax0 = fig.gca()

x2 = random.multivariate_normal([0, 0], np.diag([5**0.5, 1]), size=n_points)
x2_norm = normalise_to_axes(x2, [a, b])

ax0.scatter(*x2.T, c="C0", alpha=0.3, s=5, label="{} original points".format(n_points))
ax0.scatter(*x2_norm.T, c="C1", alpha=0.1, s=8, label = "points projected to elipse".format(n_points))
ax0.scatter([0],[0], c="C3", label="origin")
ax0.set_aspect('equal')

ax0.legend()
ax0.set_title("Projection of distribution $\sigma_a=5$, $\sigma_b=1$ \t to elipse: $a=6$, $b=1$")

Text(0.5, 1.0, 'Projection of distribution $\\sigma_a=5$, $\\sigma_b=1$ \t to elipse: $a=6$, $b=1$')

In [11]:
fig = figure()
ax0 = fig.gca()

x2 = random.multivariate_normal([0, 0], np.diag([3**0.5, 1]), size=n_points*10)
x2_norm = normalise_to_axes(x2, [a, b])

ax0.scatter(*x2.T, c="C0", alpha=0.3, s=5, label="{} original points".format(n_points))
ax0.scatter(*x2_norm.T, c="C1", alpha=0.1, s=8, label = "points projected to elipse".format(n_points))
ax0.scatter([0],[0], c="C3", label="origin")
ax0.set_aspect('equal')

ax0.legend()
ax0.set_title("Projection of distribution $\sigma_a=3$, $\sigma_b=1$ \t to elipse: $a=6$, $b=1$")

Text(0.5, 1.0, 'Projection of distribution $\\sigma_a=3$, $\\sigma_b=1$ \t to elipse: $a=6$, $b=1$')

In [10]:
at_y = attainment_sample(abs(x2_norm), n_points*2)

In [11]:
fig = figure()
ax = fig.gca()
ax.scatter(*abs(x2_norm)[:300].T, s=2)
ax.set_aspect('equal')

In [12]:
fig = figure()
ax = fig.gca()
ax.scatter(*at_y[:300].T, s=2)
ax.set_aspect('equal')

In [13]:
fig = figure()
ax0 = fig.add_subplot(2,1,1)
ax1 = fig.add_subplot(2,1,2, sharex=ax0, sharey=ax0)
ax1.set_aspect('equal')

ax0.scatter(*abs(x2_norm).T, s=5)
ax1.scatter(*at_y.T, s=5, c="C1")
ax0.set_aspect('equal')
ax0.set_xlim([0,ax0.get_xlim()[1]])

(0.0, 6.299984618698478)

In [14]:
x2_norm.shape

(10000, 2)